In [5]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sklearn

In [13]:
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [14]:
with open('intents.json') as file:
    data = json.load(file)

In [15]:
training_sentences =[]
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
num_classes = len(labels)

In [18]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels =lbl_encoder.transform(training_labels)